In [63]:
from pymongo import MongoClient
import pandas as pd
import time
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
from dotenv import load_dotenv
import os
import requests
import numpy as np
from src import get_offices_location


In [64]:
client = MongoClient("localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [65]:
db = client["ironhack"]
c = db.get_collection("companies")

In [66]:
tech_companies = db.companies.aggregate([
    {
        "$unwind": "$funding_rounds"
    },
    {
        "$group": {
            "_id": "$_id",
            "name": {
                "$first": "$name"
            },
            "raised_amount": {
                "$sum": "$funding_rounds.raised_amount"
            },
            "offices": {
                "$first": "$offices"
            },
            "category_code": {
                "$first": "$category_code"
            }
        }
    },
    {
        "$match": {
            "raised_amount": {
                "$gte": 1000000
            },
            "category_code": {
                "$in": [
                    "analytics",
                    "biotech",
                    "cleantech",
                    "ecommerce",
                    "games_video",
                    "hardware",
                    "messaging",
                    "mobile",
                    "nanotech",
                    "network_hosting",
                    "search",
                    "semiconductor",
                    "social",
                    "software",
                    "transportation",
                    "travel",
                    "web"
                ]
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "name": 1,
            "raised_amount": 1,
            "offices": 1,
            "category_code": 1
        }
    }
])

In [67]:
tech_companies_list = list(tech_companies)

In [68]:
design_companies = db.companies.aggregate([
    {
        "$match": {
            "category_code": {
                "$in": [
                    "advertising",
                    "design",
                    "fashion",
                    "photo_video"
                ]
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "name": 1,
            "offices": 1
        }
    }
])

In [69]:
design_companies_list = list (design_companies)

In [70]:
def get_offices_location (data):
    nested_data = []
    for item in data:
        name = item['name']
        for office in item['offices']:
            nested_data.append({
                'name': name,
                'office description': office['description'],
                'office latitude': office['latitude'],
                'office longitude': office['longitude']
            })
    df = pd.DataFrame(nested_data)
    return df


In [71]:
design_data = get_offices_location(design_companies_list)

In [72]:
tech_data = get_offices_location(tech_companies_list)

In [73]:
design_data = design_data.dropna(subset=["office latitude", "office longitude"], how="all")
tech_data = tech_data.dropna(subset=["office latitude", "office longitude"], how="all")

In [74]:
world_map = Map(location = [0, 0], zoom_start = 2)

In [75]:
tech_group = folium.FeatureGroup(name=f"Tech ({tech_data.shape[0]})")
design_group = folium.FeatureGroup(name = f"Design ({design_data.shape[0]})")
HeatMap(data = tech_data[["office latitude", "office longitude"]]).add_to(tech_group)
HeatMap(data = design_data[["office latitude", "office longitude"]]).add_to(design_group)
tech_group.add_to(world_map)
design_group.add_to(world_map)
folium.LayerControl(collapsed=False, position="topleft").add_to(world_map)
world_map

In [76]:
def load_cities(city_names):
    cities = {}
    for city_name in city_names:
        city_geo = gpd.read_file(f"Geojsons/{city_name}.geojson")
        cities[city_name.title()] = city_geo.geometry[0]
    return cities

def add_city_name(df, cities):
    city_names = []
    for _, row in df.iterrows():
        lat, lon = row['office latitude'], row['office longitude']
        point = gpd.points_from_xy([lon], [lat])
        for city_name, city_geometry in cities.items():
            if point.within(city_geometry)[0]:
                city_names.append(city_name)
                break
        else:
            city_names.append(None)
    df['city'] = city_names
    return df

In [77]:
city_names = ["london"]
cities = load_cities(city_names)
cities

{'London': <shapely.geometry.polygon.Polygon at 0x7ff4592cd9a0>}

In [78]:
tech_data = add_city_name(tech_data, cities)
tech_data["type"] = "tech"
tech_data

,name,office description,office latitude,office longitude,city,type
0,Sportingo,None,17.088810,-96.764920,None,tech
1,Conject,Headquarters,48.122962,11.548591,None,tech
2,Veraz Networks,,26.187571,-80.191299,None,tech
3,4Home,,37.385225,-121.991912,None,tech
4,Local Marketers,,47.598968,-122.332904,None,tech
...,...,...,...,...,...,...
3762,Winbox Technologies,Office Furtwagen (HQ),48.050514,8.205405,None,tech
3763,PodTech,None,37.408256,-122.154176,None,tech
3765,mPay Gateway,,44.989830,-93.274300,None,tech
3766,The Society,Headquarters,39.658017,-105.084475,None,tech


In [79]:
design_data = add_city_name(design_data, cities)
design_data["type"] = "design"
design_data

,name,office description,office latitude,office longitude,city,type
0,Technorati,,37.779558,-122.393041,None,design
1,AddThis,HQ - Virginia,38.926172,-77.245195,None,design
2,AddThis,New York Office,40.724604,-73.996876,None,design
3,AddThis,Los Angeles Office,34.026302,-118.380954,None,design
6,AddThis,Michigan Office,42.557958,-83.167884,None,design
...,...,...,...,...,...,...
1084,Media Whiz,HQ,26.117613,-80.263339,None,design
1085,Relevantis,,47.676786,-122.204918,None,design
1088,United On-Line,Headquarters,38.008825,23.759331,None,design
1089,MarketLine Research,,44.982264,-93.235614,None,design


In [80]:
full_city_data = pd.concat([tech_data, design_data], axis=0)
final_city = full_city_data.groupby(['city', 'type'])['name'].agg('count').reset_index()
final_city

,city,type,name
0,London,design,31
1,London,tech,52


In [81]:
final_city = full_city_data[full_city_data['city'].isin(['London'])]
final_city

,name,office description,office latitude,office longitude,city,type
29,eBuddy,Singapore office,51.500152,-0.126236,London,tech
35,Telnic,Head Office,51.518107,-0.134078,London,tech
41,WorldTV,London Office,51.500152,-0.126236,London,tech
84,WAYN,None,51.500152,-0.126236,London,tech
132,Seatwave,,51.500152,-0.126236,London,tech
...,...,...,...,...,...,...
1020,Real Time Content,London Office,51.520936,-0.142409,London,design
1036,Adeye Mobile,Adeye London HQ,51.515129,-0.129771,London,design
1042,AgreeYourFee,,51.500152,-0.126236,London,design
1063,DoTradeEasy,DoTradeEasy Ltd,51.527057,-0.104377,London,design


In [82]:
london_map = Map(location = [51, 0], zoom_start = 4)
tech_group = folium.FeatureGroup(name=f"Tech ({final_city[final_city['type'] == 'tech'].shape[0]})")
design_group = folium.FeatureGroup(name = f"Design ({final_city[final_city['type'] == 'design'].shape[0]})")


for index, row in final_city.iterrows():
    
    # 1. Marker: template
    city = {
        "location": [row["office latitude"], row["office longitude"]],
        "tooltip": row["name"]
    }
        
    # 2. icon: based on the type of company
    
    if row["type"] == "tech":
        icon = Icon (
            color = "blue",
            prefix="fa",
            icon="briefcase",
        )
    else:
        icon = Icon(
            color = "green",
            prefix="fa",
            icon="shirt"
        )
        
    
    # 3. Marker 
    new_marker = Marker (**city, icon = icon)
    
    # 4. -> to map
    if row["type"] == "tech":
        new_marker.add_to(tech_group)
    else:
        new_marker.add_to(design_group)
    

tech_group.add_to(london_map)
design_group.add_to(london_map)
folium.LayerControl(collapsed=False, position="topleft").add_to(london_map)
london_map

In [87]:
def centroid_coordinates(df):
    centroid_lat = df["office latitude"].sum() / len(df)
    centroid_lon = df["office longitude"].sum() / len(df)
    return [round(centroid_lat, 4), round(centroid_lon, 4)]

london_tech = centroid_coordinates(final_city[(final_city["city"] == 'London') & (final_city["type"] == 'tech')])
london_design = centroid_coordinates(final_city[(final_city["city"] == 'London') & (final_city["type"] == 'design')])


In [88]:
def add_marker (name, color, icon_, coordinates, map):
    icon1 = Icon(
    color = color,
    opacity = 0.1,
    prefix = "fa", #font-awesome
    icon = icon_,
    icon_color = "white"
    )   
    marker_ = Marker(
    location = coordinates,
    tooltip = name,
    icon = icon1
    )
    marker_.add_to(map)
    return map


In [89]:
add_marker("tech_middle","red","rocket",london_tech,london_map)
add_marker("design_middle","red","rocket",london_design,london_map)

In [90]:
def mean_coordinates_raw (list_):
    x = 0
    y = 0
    for i in list_:
        x +=i[0]
        y +=i[1]
    x = x/len(list_)
    y = y/len(list_)

    return [round(x,4), round(y,4)]

office_location = mean_coordinates_raw([london_tech,london_design])
office_location

[51.5134, -0.119]

In [91]:
load_dotenv()

True

In [92]:
fsq_tok = os.getenv("token")

In [93]:
def process_4sq_data(data):
    rows = []
    for result in data['results']:
        row = {
            'name': result['name'],
            'distance': result['distance'],
            'latitude': result['geocodes']['main']['latitude'],
            'longitude': result['geocodes']['main']['longitude'],
            'category_id': result['categories'][0]['id'],
            'category_name': result['categories'][0]['name']
        }
        rows.append(row)
    df = pd.DataFrame(rows).sort_values(by='distance', ascending=True)
    return df

In [94]:
url = "https://api.foursquare.com/v3/places/search?ll=51.5134%2C-0.1191&radius=50000&categories=19040"

headers = {
    "accept": "application/json",
    "Authorization": fsq_tok
}

response = requests.get(url, headers=headers)

airports_raw = response.json()
airports = process_4sq_data(airports_raw)
airports


,name,distance,latitude,longitude,category_id,category_name
1,London City Airport (LCY),11710,51.504029,0.049552,19040,International Airport
0,London Heathrow Airport (LHR),24005,51.470584,-0.454958,19040,International Airport
2,London Gatwick Airport (LGW),39601,51.156710,-0.163651,19040,International Airport


In [95]:
url = "https://api.foursquare.com/v3/places/search?ll=51.5134%2C-0.1191&radius=2000&categories=12056%2C12057&sort=DISTANCE&limit=30"

headers = {
    "accept": "application/json",
    "Authorization": fsq_tok
}

response_schools = requests.get(url, headers=headers)

schools_raw = response_schools.json()
schools = process_4sq_data(schools_raw)
schools


,name,distance,latitude,longitude,category_id,category_name
0,St Clement Danes C of E Primary School,53,51.513551,-0.119217,12058,Elementary School
1,St Josephs R C Primary School,361,51.516129,-0.121953,12058,Elementary School
2,Kensington Aldridge Academy,608,51.508648,-0.123480,12059,High School
3,Erp Solution Advisors,852,51.517171,-0.129837,12057,Primary and Secondary School
4,St Albans Primary School,960,51.520223,-0.110579,12058,Elementary School
5,St George the Martyr Church of England Primary...,977,51.522017,-0.116266,12057,Primary and Secondary School
6,Rubadubs Nursery Limited,984,51.518732,-0.107732,12056,Preschool
7,Jivjav,1021,51.507478,-0.107800,12057,Primary and Secondary School
8,London Nautical School,1057,51.506494,-0.108033,12059,High School
9,Soho Parish C of E Primary School,1059,51.511711,-0.134438,12058,Elementary School


In [96]:
url = "https://api.foursquare.com/v3/places/search?ll=51.5134%2C-0.1191&radius=2000&chains=ab4c54c0-d68a-012e-5619-003048cad9da&sort=DISTANCE&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": fsq_tok
}

response_starbucks = requests.get(url, headers=headers)

starbucks_raw = response_starbucks.json()
starbucks = process_4sq_data(starbucks_raw)
starbucks

,name,distance,latitude,longitude,category_id,category_name
0,Starbucks,111,51.513847,-0.117479,13035,Coffee Shop
1,Starbucks,214,51.512244,-0.121751,13035,Coffee Shop
2,Starbucks,332,51.516379,-0.120061,13035,Coffee Shop
3,Starbucks,546,51.514722,-0.126655,13035,Coffee Shop
4,Starbucks,700,51.507587,-0.123026,13035,Coffee Shop
5,Starbucks,710,51.507480,-0.122999,13035,Coffee Shop
6,Starbucks,714,51.518223,-0.125809,13035,Coffee Shop
7,Starbucks,740,51.518523,-0.112254,13035,Coffee Shop
8,Starbucks,793,51.515096,-0.130274,13035,Coffee Shop
9,Starbucks,795,51.516658,-0.129233,13035,Coffee Shop


In [97]:
url = "https://api.foursquare.com/v3/places/search?ll=51.5134%2C-0.1191&radius=10000&categories=18006&sort=DISTANCE&limit=15"

headers = {
    "accept": "application/json",
    "Authorization": fsq_tok
}

response_basket = requests.get(url, headers=headers)

basket_raw = response_basket.json()
basket = process_4sq_data(basket_raw)
basket


,name,distance,latitude,longitude,category_id,category_name
0,Basket Ball Court,1462,51.525298,-0.109598,18008,Basketball Court
1,Spa Fields Basketball Court,1504,51.525410,-0.108339,18008,Basketball Court
2,GMH Park Netball Courts,2127,51.496010,-0.106626,18008,Basketball Court
3,The Regal,2460,51.491352,-0.114949,18008,Basketball Court
4,Basketball Court,2632,51.501277,-0.086212,18008,Basketball Court
5,Netball Courts,2816,51.536528,-0.137030,18008,Basketball Court
6,Basketball Court,3397,51.531601,-0.079473,18008,Basketball Court
7,Lisson Grove - Basketball Court,3668,51.525673,-0.168182,18008,Basketball Court
8,Basketball Courts,3707,51.487366,-0.085247,18008,Basketball Court
9,Tabard Gardens - Basketball Court,3852,51.514811,-0.174869,18008,Basketball Court


In [98]:
url = "https://api.foursquare.com/v3/places/search?ll=51.5134%2C-0.1191&radius=5000&categories=13377&sort=DISTANCE&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": fsq_tok
}

response_vegan = requests.get(url, headers=headers)

vegan_raw = response_vegan.json()
vegan = process_4sq_data(vegan_raw)
vegan



,name,distance,latitude,longitude,category_id,category_name
0,Sagar,141,51.512617,-0.120700,13072,Asian Restaurant
1,Farmstand,224,51.514349,-0.121845,13377,Vegan and Vegetarian Restaurant
2,Just Falafs,308,51.512025,-0.122983,13145,Fast Food Restaurant
3,Wild Food Cafe,519,51.514521,-0.126319,13377,Vegan and Vegetarian Restaurant
4,Pilpel,828,51.514372,-0.107270,13144,Falafel Restaurant
5,Vitao,914,51.516507,-0.131340,13377,Vegan and Vegetarian Restaurant
6,Vantra Loungevity,935,51.510579,-0.132012,13377,Vegan and Vegetarian Restaurant
7,Govinda's Restaurant,973,51.515847,-0.132749,13199,Indian Restaurant
8,Soho Vegan Market,1021,51.512346,-0.133595,13377,Vegan and Vegetarian Restaurant
9,Leon,1027,51.514310,-0.103920,13145,Fast Food Restaurant


In [99]:
url = "https://api.foursquare.com/v3/places/search?ll=51.5134%2C-0.1191&radius=2500&categories=11134&sort=DISTANCE&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": fsq_tok
}

response_pet = requests.get(url, headers=headers)
pet_raw = response_pet.json()
pet = process_4sq_data(pet_raw)
pet

,name,distance,latitude,longitude,category_id,category_name
0,BOW WOW London Dog Grooming,661,51.518280,-0.124584,11134,Pet Grooming Service
1,Marabese,1685,51.500761,-0.132576,11134,Pet Grooming Service
2,Love Your Pets Grooming & Boutique,2288,51.533094,-0.109435,11134,Pet Grooming Service


In [100]:
url = "https://api.foursquare.com/v3/places/search?ll=51.5134%2C-0.1191&radius=1000&categories=10032"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3BsavsPN2+4+aYMMFcu7hhMBtnzwJ0a3poXWeA0zd5dg="
}

response_club= requests.get(url, headers=headers)
club_raw = response_club.json()
clubs = process_4sq_data(club_raw)
clubs

,name,distance,latitude,longitude,category_id,category_name
9,The Hercules Pillars,290,51.515856,-0.120837,10032,Night Club
5,Proud Embankment,386,51.510094,-0.117930,10032,Night Club
8,The Yacht London,472,51.510478,-0.114309,10032,Night Club
2,Lucky Voice Holborn,535,51.515777,-0.112002,10032,Night Club
0,Retro Bar,562,51.509123,-0.123740,10032,Night Club
6,Heaven,695,51.508129,-0.124161,10032,Night Club
4,Ku Bar,745,51.511827,-0.129606,10032,Night Club
1,Ronnie Scott's,867,51.513310,-0.131451,10032,Night Club
3,Duke of Wellington,976,51.512307,-0.133251,10032,Night Club
7,The Friendly Society,982,51.512363,-0.133218,10032,Night Club


In [101]:
pet["group"] = "Pet Grooming"
vegan["group"] = "Vegan Food"
basket["group"] = "Basket court"
starbucks["group"] = "Starbucks"
schools["group"] = "Schools"
airports["group"] = "Airports"
clubs["group"]= "Clubs"

In [102]:
near_office = pd.concat([pet, vegan, basket, starbucks, schools, airports, clubs], axis=0)

In [103]:
near_office

,name,distance,latitude,longitude,category_id,category_name,group
0,BOW WOW London Dog Grooming,661,51.518280,-0.124584,11134,Pet Grooming Service,Pet Grooming
1,Marabese,1685,51.500761,-0.132576,11134,Pet Grooming Service,Pet Grooming
2,Love Your Pets Grooming & Boutique,2288,51.533094,-0.109435,11134,Pet Grooming Service,Pet Grooming
0,Sagar,141,51.512617,-0.120700,13072,Asian Restaurant,Vegan Food
1,Farmstand,224,51.514349,-0.121845,13377,Vegan and Vegetarian Restaurant,Vegan Food
...,...,...,...,...,...,...,...
6,Heaven,695,51.508129,-0.124161,10032,Night Club,Clubs
4,Ku Bar,745,51.511827,-0.129606,10032,Night Club,Clubs
1,Ronnie Scott's,867,51.513310,-0.131451,10032,Night Club,Clubs
3,Duke of Wellington,976,51.512307,-0.133251,10032,Night Club,Clubs


In [104]:
office_nearby = Map(location = [51, 0], zoom_start = 11.4)
pet_group = folium.FeatureGroup(name=f"Pet grooming ({near_office[near_office['group'] == 'Pet Grooming'].shape[0]})")
vegan_group = folium.FeatureGroup(name = f"Vegan restaurants ({near_office[near_office['group'] == 'Vegan Food'].shape[0]})")
basket_group = folium.FeatureGroup(name = f"Basketball courts ({near_office[near_office['group'] == 'Basket court'].shape[0]})")
starbucks_group = folium.FeatureGroup(name = f"Starbucks ({near_office[near_office['group'] == 'Starbucks'].shape[0]})")
schools_group = folium.FeatureGroup(name = f"Schools ({near_office[near_office['group'] == 'Schools'].shape[0]})")
airports_group = folium.FeatureGroup(name = f"Airports ({near_office[near_office['group'] == 'Airports'].shape[0]})")
clubs_group = folium.FeatureGroup(name = f"Clubs ({near_office[near_office['group'] == 'Clubs'].shape[0]})")

# Iteration through DataFrame to create marker and add it to the corresponding group.
for index, row in near_office.iterrows():
    
    # 1. Marker: creates the marker in the office location and adds the name to it.
    city = {
        "location": [row["latitude"], row["longitude"]],
        "tooltip": row["name"]
    }
        
    # 2. Add the icon: based on the type of venue
    
    if row["group"] == "Pet Grooming":
        icon = Icon (
            color = "blue",
            prefix="fa",
            icon="dog"
        )
    elif row["group"] == "Vegan Food":
        icon = Icon (
            color = "lightgreen",
            prefix="fa",
            icon="leaf"
        )
    elif row["group"] == "Basket court":
        icon = Icon (
            color = "orange",
            prefix="fa",
            icon="basketball"
        )
    elif row["group"] == "Starbucks":
        icon = Icon (
            color = "green",
            prefix="fa",
            icon="coffee"
        )
    elif row["group"] == "Schools":
        icon = Icon (
            color = "purple",
            prefix="fa",
            icon="school"
        )
    elif row["group"] == "Clubs":
        icon = Icon (
            color = "darkblue",
            prefix="fa",
            icon="martini-glass"
        )
    else:
        icon = Icon(
            color = "cadetblue",
            prefix="fa",
            icon="plane"
        )
        
    
    # 3. Creates the map Marker
    new_marker = Marker (**city, icon = icon)
    
    # 4. Adds the marker to the corresponding group
    if row["group"] == "Pet Grooming":
        new_marker.add_to(pet_group)
    elif row["group"] == "Vegan Food":
        new_marker.add_to(vegan_group)
    elif row["group"] == "Basket court":
        new_marker.add_to(basket_group)
    elif row["group"] == "Starbucks":
        new_marker.add_to(starbucks_group)
    elif row["group"] == "Schools":
        new_marker.add_to(schools_group)
    elif row["group"] == "Clubs":
        new_marker.add_to(clubs_group)
    else:
        new_marker.add_to(airports_group)

# Now we add the groups to the maps
pet_group.add_to(office_nearby)
vegan_group.add_to(office_nearby)
basket_group.add_to(office_nearby)
starbucks_group.add_to(office_nearby)
schools_group.add_to(office_nearby)
airports_group.add_to(office_nearby)
clubs_group.add_to(office_nearby)


# Add the office
add_marker("Office Location","red","computer",office_location,office_nearby)

# Add the llayer control
folium.LayerControl(collapsed=False, position="topleft").add_to(office_nearby)
office_nearby

